<a href="https://colab.research.google.com/github/PhoenixStormJr/Unfinished-Colab-AI-enhance-video/blob/main/whisper_upload_translated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Go to Google Drive and upload the file to the root of the drive

https://drive.google.com/drive/

In [ ]:
#@title ##**Video file name** { display-mode: "form" }
file_name = input("Enter file name: ") or "input.mp3"
file_name_audio = str(file_name)

In [ ]:
#@title ##**Install dependencies** { display-mode: "form" }
!git clone https://huggingface.co/spaces/openai/whisper
%cd whisper
!pip install -r requirements.txt
!pip install gradio
!pip install pysrt
import pysrt

In [ ]:
#@title ##**Connecting to Google Drive** { display-mode: "form" }
from google.colab import drive
drive.mount('/mnt/gdrive')


In [ ]:
#@title ##**Split file** { display-mode: "form" }
from pydub import AudioSegment
from pydub.utils import make_chunks
import os
import shutil

split = "auto" #@param ["30_second","auto"]
chunk_folder = "/content/whisper/audio_chunks"
if os.path.isdir(chunk_folder):
    shutil.rmtree(chunk_folder)

if (split == "30_second"):

  path = '/mnt/gdrive/MyDrive/'
  full_path = os.path.join(path, file_name)

  def split_audio_file(audio_file_path, chunk_length_ms=30000):

      audio = AudioSegment.from_file(audio_file_path)
      audio_chunks = make_chunks(audio, chunk_length_ms)

      os.makedirs(chunk_folder, exist_ok=True)

      for i, chunk in enumerate(audio_chunks):
          chunk_name = f'{chunk_folder}/chunk_{i:04d}.wav'
          print(chunk_name)
          chunk.export(chunk_name, format="wav")

      return chunk_folder

  split_audio_file(full_path)

if (split == "auto"):
  from librosa.util.utils import normalize
  import os
  import librosa
  import numpy as np
  import soundfile as sf
  import shutil
  import audioread

  import scipy.io.wavfile as wav
  import scipy.signal as signal
  from scipy.signal import find_peaks
  from scipy.io import wavfile
  import moviepy.editor as mp
  from scipy.io import loadmat


  chunk_folder = "/content/whisper/audio_chunks"
  if os.path.isdir(chunk_folder):
      shutil.rmtree(chunk_folder)
  os.makedirs(chunk_folder)
  path = '/mnt/gdrive/MyDrive/'

  full_path = os.path.join(path, file_name)

  def convert_to_wav(input_file):
      file_name, ext = os.path.splitext(input_file)
      if ext == '.wav':
          return input_file

      output_file = file_name + '.wav'
      sound = AudioSegment.from_file(input_file, format=ext[1:])
      sound.export(output_file, format='wav')
      return output_file

  full_path = convert_to_wav(full_path)
  print(full_path)
  # Upload an audio file and get signal data
  rate, data = wavfile.read(full_path)

  # Normalize audio to -3 dB
  target_db = -3.0
  max_val = np.iinfo(data.dtype).max
  target_val = np.power(10.0, target_db / 20.0) * max_val
  data = data.astype(np.float64) * target_val / np.max(np.abs(data))
  data = data.astype(np.int16)

  # Save normalized audio file
  normalize_file = f"/mnt/gdrive/MyDrive/normalized_{file_name}"
  wavfile.write(normalize_file, rate, data)

  def split_chunk(input_file, output_dir, intervals_ms):
      # Open an audio file and load its content and sample rate
      audio, sr = sf.read(input_file)

      # Split an audio file into separate parts
      for i in range(len(intervals_ms) - 1):
          start_ms = intervals_ms[i]
          end_ms = intervals_ms[i+1]

          # Calculate start and end indexes in samples
          start = int(start_ms / 1000 * sr)
          end = int(end_ms / 1000 * sr)

          # Write a separate part of an audio file to a new file
          output_file = f'{chunk_folder}/chunk_{i:04d}.wav'
          sf.write(output_file, audio[start:end], sr)


  def split_audio(input_file, output_dir, min_duration=1.0, max_duration=10.0, silence_threshold=0.5, min_length=5):
      # Upload an audio file and get signal data
      audio_file, sr = librosa.load(input_file, sr=None, mono=True)

      # Calculate frame duration and frame size
      frame_duration = 0.025
      frame_len = int(frame_duration * sr)

      # Calculate the threshold value of the energy signal to determine silence
      peak_threshold = np.max(audio_file) * silence_threshold

      # Calculate energy signal
      audio_file = np.concatenate((audio_file, np.zeros(frame_len - len(audio_file) % frame_len)))
      energies = np.sum(np.square(audio_file.reshape(-1, frame_len)), axis=1)

      # Find peaks in energy signal
      peak_indices, _ = find_peaks(energies, height=peak_threshold)

      # Create an array of silence intervals
      silence_intervals = []

      # If the first peak is not at the beginning of the signal, add an interval from the beginning to the first peak
      if peak_indices[0] != 0:
          silence_intervals.append((0, peak_indices[0] * frame_len))

      # Add silence intervals between peaks if the interval length is greater than min_duration
      for i in range(len(peak_indices) - 1):
          start = peak_indices[i] * frame_len
          end = peak_indices[i+1] * frame_len
          duration = (end - start) / sr
          if duration > min_duration:
              silence_intervals.append((start, end))

      # Add the last silence interval if its length is greater than min_duration and less than or equal to max_duration
      if peak_indices[-1] * frame_len != len(audio_file) - 1:
          start = peak_indices[-1] * frame_len
          end = len(audio_file) - 1
          duration = (end - start) / sr
          if duration > min_duration and duration <= max_duration:
              silence_intervals.append((start, end))

      silence_intervals = [(start * 1000 / sr, end * 1000 / sr) for start, end in silence_intervals]

      means = []
      for start, end in silence_intervals:
          mean = (start + end) / 2
          means.append(mean)
      # calculate the length of the audio file in milliseconds and add it to the end of the intervals_ms list
      file_duration_ms = np.round(len(audio_file) / sr * 1000).astype(int)
      means = np.append(means, file_duration_ms)
      means = np.insert(means, 0, 0)
      means = means.astype(int)
      i = 0
      min_interval = 15000
      max_interval = 30000
      while i < len(means) - 1:
          # check if next element is less than current element + 15
          if means[i+1] < means[i] + min_interval:
              #@markdown remove next element
              means = np.delete(means, i+1)
          # check if next element is equal to current element + 15 and less than current element + 30
          elif means[i+1] == means[i] + min_interval and means[i+1] < means[i] + max_interval:
              #@markdown leave the next element and move on to the next iteration
              i += 1
          # check if next element is greater than current element + 30
          elif means[i+1] > means[i] + max_interval:
              # add a new element that will be equal to the current one + 30 and move on to the next iteration
              means = np.insert(means, i+1, means[i]+max_interval)
              i += 1
          else:
              i += 1
      means = np.append(means, file_duration_ms)
      print(silence_intervals)
      print(means)
      split_chunk(full_path, chunk_folder, means)
      return means

  time_labels = split_audio(normalize_file, chunk_folder)

In [ ]:
#@title ##**Model installation** { display-mode: "form" }
import os
os.system("pip install git+https://github.com/openai/whisper.git")
import gradio as gr
import whisper

from share_btn import community_icon_html, loading_icon_html, share_js

model = whisper.load_model("large-v2")




In [ ]:
#@title ##**Launch** { display-mode: "form" }
import os

import pysrt
import datetime

result_subs = pysrt.SubRipFile()

import datetime
import pysrt

def create_subtitles(start_time, end_time, text):
    subs = pysrt.SubRipFile()
    start_time = datetime.timedelta(milliseconds=int(start_time))
    end_time = datetime.timedelta(milliseconds=int(end_time))
    start_time_dict = {'hours': 0, 'minutes': 0, 'seconds': start_time.seconds, 'milliseconds': start_time.microseconds // 1000}
    end_time_dict = {'hours': 0, 'minutes': 0, 'seconds': end_time.seconds, 'milliseconds': end_time.microseconds // 1000}
    subs.append(pysrt.SubRipItem(index=1, start=pysrt.SubRipTime(**start_time_dict), end=pysrt.SubRipTime(**end_time_dict), text=text))
    return subs



def inference(audio):
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    _, probs = model.detect_language(mel)

    options = whisper.DecodingOptions(fp16 = False)
    result = whisper.decode(model, mel, options)

    print(result.text)
    return result.text, gr.update(visible=True), gr.update(visible=True), gr.update(visible=True)

# Processing each piece and writing the result to a file
result_text = ""
chunk_folder = "/content/whisper/audio_chunks"
chunk_files = sorted(os.listdir(chunk_folder))
for i in range(len(chunk_files)-1):
    file_name = chunk_files[i]
    if file_name.endswith('.wav'):
        file_path = os.path.join(chunk_folder, file_name)
        text = inference(file_path)[0]
        result_text += text

        start_time = time_labels[i]
        end_time = time_labels[i+1] if i+1<len(time_labels) else start_time + 5000
        subs = create_subtitles(start_time, end_time, text)
        result_subs.extend(subs)

# Saving the result to a file
with open('result.txt', 'w') as f:
    f.write(result_text)
name_without_extension = os.path.splitext(file_name_audio)[0]
path_str = "/content/whisper/"+name_without_extension+".srt"
result_subs.save(path_str)

In [ ]:
#@title ##**Download result** { display-mode: "form" }
import re

with open('result.txt', 'r') as f:
    text = f.read()

# replace the end of the sentence with a line break
text = re.sub(r'([.?!])\s+', r'\1\n', text)

with open('result.txt', 'w') as f:
    f.write(text)

from google.colab import files

# loading result.txt file into Colab
files.download('result.txt')

In [ ]:
#@title ##**Download subtitles** { display-mode: "form" }
from google.colab import files
files.download(path_str)